In [1]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import tqdm

In [2]:
path = r"C:\Users\Acer\Git\deda_punks\Crypto_punks\bids"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
punks = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            punks.append(file.name)

In [3]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [4]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [5]:
data = {}
p = r"C:\Users\Acer\Git\deda_punks\punks_features.pkl"

# lop through each image in the dataset
for punk in tqdm.tqdm(punks):
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(punk,model)
        data[punk] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)

100%|████████████████████████████████████████████████████████████████████████████████| 908/908 [07:06<00:00,  2.13it/s]


In [6]:
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
print(feat.shape)

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
print(feat.shape)

(908, 1, 4096)
(908, 4096)


In [36]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

In [42]:
kmeans = KMeans(n_clusters=100,n_jobs=-1, random_state=22)
kmeans.fit(x)

c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


KMeans(n_clusters=100, n_jobs=-1, random_state=22)

In [43]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [44]:
groups[0]

['Punk 0105.png',
 'Punk 0738.png',
 'Punk 0798.png',
 'Punk 1170.png',
 'Punk 1290.png',
 'Punk 2219.png',
 'Punk 2842.png',
 'Punk 2912.png',
 'Punk 2987.png',
 'Punk 3643.png',
 'Punk 3886.png',
 'Punk 5557.png',
 'Punk 5701.png',
 'Punk 6188.png',
 'Punk 9062.png',
 'Punk 9726.png']

In [45]:
display(img)

In [62]:
from PIL import Image

for mini_punk in groups[12]:
    img = Image.open(r"C:\Users\Acer\Git\deda_punks\Crypto_punks\bids\\"+mini_punk)
    display(img)